# SQL query from table names - Continued

In [1]:
pip install dotenv

In [9]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('')

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [4]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [5]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [13]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=""
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [14]:
#new
context_user = context.copy()
print(return_CCRMSQL("What are the names of employees who studied at MIT and earned more than 80000 in the year 2022?", context_user))


```sql
SELECT e.name
FROM employees e
JOIN education ed ON e.employee_id = ed.employee_id
JOIN salary s ON e.employee_id = s.employee_id
WHERE ed.university = 'MIT' AND s.salary > 80000 AND s.year = 2022;
```


In [15]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("What are the names of employees who studied at MIT and earned more than 80000 in the year 2022?", old_context_user))


This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
WHERE s.Institution = 'MIT' AND sa.salary > 80000 AND YEAR(sa.year) = 2022;
```

This SQL query retrieves the names of employees who studied at MIT and earned more than 80000 in the year 2022. It joins the employees, studies, and salary tables on the corresponding ID columns, filters the results based on the conditions specified, and selects the names of the employees meeting the criteria.


In [16]:
#new
print(return_CCRMSQL("What are the names of employees who studied at MIT and earned more than 80000 in the year 2022?", context_user))


```sql
SELECT e.name
FROM employees e
JOIN education ed ON e.employee_id = ed.employee_id
JOIN salary s ON e.employee_id = s.employee_id
WHERE ed.university = 'MIT' AND s.salary > 80000 AND s.year = 2022;
```


In [17]:
#old
print(return_CCRMSQL("What are the names of employees who studied at MIT and earned more than 80000 in the year 2022?", old_context_user))


This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
WHERE s.Institution = 'MIT' AND sa.salary > 80000 AND YEAR(sa.year) = 2022;
```

This SQL query retrieves the names of employees who studied at MIT and earned more than 80000 in the year 2022. It achieves this by joining the three tables (employees, studies, and salary) on the corresponding ID columns and then filtering the results based on the specified conditions.


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [18]:
print(return_CCRMSQL("What are the names of employees who studied at MIT and earned more than 80000 in the year 2022?", old_context_user))


This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
WHERE s.Institution = 'MIT' AND sa.salary > 80000 AND YEAR(sa.year) = 2022;
```

This SQL query retrieves the names of employees who studied at MIT and earned more than 80000 in the year 2022. It joins the employees, studies, and salary tables on the corresponding ID columns, filters the results based on the conditions specified, and selects the names of the employees meeting the criteria.


In [19]:
print(return_CCRMSQL("What are the names of employees who studied at MIT and earned more than 80000 in the year 2022?", context_user))


```sql
SELECT e.name
FROM employees e
JOIN education ed ON e.employee_id = ed.employee_id
JOIN salary s ON e.employee_id = s.employee_id
WHERE ed.university = 'MIT' AND s.salary > 80000 AND s.year = 2022;
```


In [20]:
print(return_CCRMSQL(
    "Your goal is to generate an SQL query that returns the names of employees who: "
    "- studied at MIT, "
    "- earned more than 80000, "
    "- in the year 2022. "
    "Use the correct joins and filter conditions based on the table schema."
, old_context_user))


This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
JOIN salary sa ON e.ID_usr = sa.ID_usr
WHERE s.Institution = 'MIT' AND sa.salary > 80000 AND YEAR(sa.year) = 2022;
```

This SQL query selects the names of employees who studied at MIT, earned more than 80000 in the year 2022 by joining the three tables (employees, studies, and salary) based on the common ID_usr field and applying the specified filter conditions.




In this exercise, we tested three different prompt styles to generate SQL queries using GPT-3.5. The question was: "What are the names of employees who studied at MIT and earned more than 80000 in the year 2022?" The first version (old prompt) gave the model a basic description of the tables using plain language. It did okay by joining the correct tables and filtering for salaries over 80000, but it missed the year 2022 filter. This shows the model didn’t fully understand all the conditions without a structured format or examples. The second version (new prompt) followed a paper from Ohio University. It gave the model proper SQL structure using CREATE TABLE for each table and added few-shot examples of SQL queries. This helped a lot. The model gave a correct SQL query with all conditions, including the correct date filter using strftime. The third version was a small improvement to the second one. We changed the few-shot examples to include a query with a date condition. This helped the model generalize the idea better. The output was accurate again. From this experiment, we learned that few-shot examples are very useful in helping the model write better SQL. Adding CREATE TABLE structure makes it easier for the model to understand the database schema. Also, including a sample that filters by date helps the model apply that pattern in other queries. On the other hand, the old prompt without examples didn’t work well. It led to missing important parts of the question, like the year filter. In short, prompt design matters a lot when doing natural language to SQL tasks. Even small changes in prompt structure or examples can make a big difference in the final result. It’s always a good idea to try different versions and test which one gives the most reliable and accurate output.